In [1]:
import tellurium as te
import pandas as pd
import numpy as np

import cobra
import os
os.getcwd()


from src.no_BayesianInference import BMCA, util

In [2]:
os.getcwd()

'c:\\Users\\user\\Documents\\research\\BMCA-pipeline'

Generating data for running through the pipeline.

Running BMCA on BioModel 64

In [7]:
# load the Teusink model
ant_file = 'data/interim/Antimony/BIOMD0000000064_e.ant'
data_file = 'data/interim/generated_data/BIOMD0000000064_e_50.csv'
data = pd.read_csv(data_file)

Calculating the steady state concentrations of the Teusink model when EtOH concentration is halved. 

In [24]:
s  = te.loada(ant_file)
s.ETOH = s.ETOH/2
s.steadyState()
true_ss_values = pd.DataFrame(s.getFloatingSpeciesConcentrations(), index=s.getFloatingSpeciesIds(), columns=['true'])

# s.getScaledFluxControlCoefficientMatrix()

Predicting the steady state concentration values 

In [25]:
BIOMD64 = BMCA.BMCA(ant_file, data_file, desired_product='ETOH')

# pd.DataFrame(pipeline.Ex, columns=s.getFloatingSpeciesIds(), index=s.getReactionIds())
# pd.DataFrame(pipeline.Ey, columns=s.getBoundarySpeciesIds(), index=s.getReactionIds())

chi, v_hat = BIOMD64.calculate_steady_state(BIOMD64.Ex, BIOMD64.Ey)

estimated_ss = pd.DataFrame(data=chi.to_numpy(), index=s.getFloatingSpeciesIds(), columns=['estimated'])

c:\Users\user\Documents\research\BMCA-pipeline\src\no_BayesianInference\BMCA.py:141: FutureWarning:

Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.



In [26]:
comparison = pd.concat([true_ss_values, estimated_ss], axis = 1)
comparison['error'] = (comparison['true']-comparison['estimated'])/comparison['true']

In [27]:
comparison

,true,estimated,error
P,6.769148,-0.481900,1.071191
G6P,1.283076,-0.374763,1.292082
TRIO,0.814107,0.036631,0.955005
NAD,1.559295,0.037061,0.976232
ACE,0.136863,0.224228,-0.638331
P2G,0.054551,0.271243,-3.972267
BPG,0.000510,0.733958,-1437.404301
GLCi,0.086895,0.785709,-8.042036
F6P,0.156413,0.837461,-4.354155
PEP,0.099611,1.300176,-12.052562


In [21]:
comparison

,0,0
P,6.769148,-0.481900
G6P,1.283076,-0.374763
TRIO,0.814107,0.036631
NAD,1.559295,0.037061
ACE,0.136863,0.224228
P2G,0.054551,0.271243
BPG,0.000510,0.733958
GLCi,0.086895,0.785709
F6P,0.156413,0.837461
PEP,0.099611,1.300176
